In [1]:
import pandas as pd
import numpy as np
import math as m
import time as t

In [2]:
# 3 Main Steps
# 1. Produce list of consecutive primes
# 2. Produce list of sums of consecutive primes under one million
# 3. Check if the sum for the longest list is prime, and continue 
#    until longest list is found

In [3]:
# Originally used Wilson's Theorem to check for a prime
# But quickly realized this was computationally intensive
# As it required the calculation (n-1)!
# More cost efficient to check for all possible factors

#Test if Prime Function
def PrimeTest(n) :
    if n%2 == 0:
        return False
    for i in range(3,m.ceil(m.sqrt(n))+2,2):
        if n%i == 0 :
            return False
    return True      

In [5]:
# Generate List of Consecutive Primes
# To reduce the calls of PrimeTest, I implemented the rule
# that all primes are either 6n-1 or 6n+1 
# (Removing all multiples of 2 and 3) 
# We append 2 and 3 at the initialization of the list
# as these are the only primes that do not follow this 
# convention

primeslist = np.zeros(1000, dtype=int)
primeslist[0] = 2
primeslist[1] = 3
count = 2;

for n in range(1, 1000) :
    if PrimeTest(6*n-1) == True :
        primeslist[count] = int(6*n-1)
        count +=1
    if PrimeTest(6*n+1) == True :
        primeslist[count] = int(6*n+1)
        count +=1

primes = primeslist[primeslist != 0]
primes.sort()

# We see the longest list of consecutive primes we have is 783
# As well as the total sum of that list, which is well above 1 million
print(len(primes))
print(np.sum(primes))

783
2174734


In [6]:
# We now create a matrix of the sums for all possible 
# series of consecutive primes starting with the longest list
# We begin with the sum of our entire prime series then reduce
# the size by one. We now have two possible sums of this length
# as we can shift the series by one step (excluding the last or first term)
# We iterate through this process, decreasing the series length by 1
# and keep track of all sums below 1 million. We could repeat this 
# process down to series of 2 but we set a stopping condition of
# 400 consecutive primes as a starting point. If the solution is
# not found we would decrease that stopping condition.

nump = len(primes)
maxseries = nump
minseries = 400
testsumarray = np.zeros((nump**2,2))
count=0

for size in range(maxseries,minseries,-1) :
    for shift in range(0,nump-size) :
        testsum = np.sum(primes[shift:size+shift])
        #print(testsum)
        if testsum >= 10**6 :
            break
        else :
            testsumarray[count,0] = size
            testsumarray[count,1] = testsum
            count += 1

            np.max(testsumarray[0,:])
testsumarray = testsumarray[testsumarray[:,0] !=0]

In [7]:
# Putting the sums into a dataframe, we see that our longest series is 546
# with a sum of 997661. Second longest series is 545 with two possible sums 
# below a million, one is obviously not prime as it is even
# We now must check which sum is prime

sumdf = pd.DataFrame(testsumarray, columns =['Series Length','Prime Sum'])
sumdf = sumdf.sort_values(by=['Series Length','Prime Sum'], ascending = False)
sumdf

,Series Length,Prime Sum
0,546.0,997661.0
2,545.0,997659.0
1,545.0,993730.0
5,544.0,997656.0
4,544.0,993728.0
...,...,...
11342,401.0,521643.0
11341,401.0,518861.0
11340,401.0,516089.0
11339,401.0,513325.0


In [8]:
# Starting with the longest series and highest sum, we check if the sum is prime
# In order to reduce execution of PrimeTest, we do a quick check with modular
# division to make sure the sum is 6n-1 or 6n+1
# We repeate this process until the highest sum of the longest series is found

complete = 0
trynum = 0
print('First Try:',sumdf.iloc[trynum,1],' Series Length:',sumdf.iloc[trynum,0])

while complete == 0 :
    if (int(sumdf.iloc[trynum,1])-1)%6==0 or (int(sumdf.iloc[trynum,1])+1)%6==0 :
        if PrimeTest(int(sumdf.iloc[trynum,1])) == True :
            complete = 1
        else :
            trynum += 1
            print('Next Try:',sumdf.iloc[trynum,1],' Series Length:',sumdf.iloc[trynum,0])
    else:
        trynum += 1
        print('Next Try:',sumdf.iloc[trynum,1],' Series Length:',sumdf.iloc[trynum,0])    
else :
    print('Highest Prime:',sumdf.iloc[trynum,1],' Series Length:',sumdf.iloc[trynum,0])

First Try: 997661.0  Series Length: 546.0
Next Try: 997659.0  Series Length: 545.0
Next Try: 993730.0  Series Length: 545.0
Next Try: 997656.0  Series Length: 544.0
Next Try: 993728.0  Series Length: 544.0
Next Try: 989801.0  Series Length: 544.0
Next Try: 997651.0  Series Length: 543.0
Highest Prime: 997651.0  Series Length: 543.0
